In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import *
from IPython.display import display, HTML, IFrame
%matplotlib inline
from collections import defaultdict
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png','pdf')


In [2]:
#folder = ['compsci-data','jbt-data', 'brands-data-400', 'brands-data-100', 'brands-data-it2', 'jbt-data-it2', 'compsci-data-it2']
MERGE_OTHER = True
ut = 'jbt'
folder = ['{}-data-it2'.format(ut), '{}-data'.format(ut)]
#folder = ['{}-data-it3'.format(ut)]

In [3]:


agg_list = []
full_list = []
test_list = []
for f in folder:
    agg =  pd.read_csv('{}/agg.csv'.format(f))
    agg['type'] = f.split('-')[0]
    agg.set_index('id')
    agg_list.append(agg)
   
    full = pd.read_csv('{}/full.csv'.format(f))
    full['type'] = f.split('-')[0]
    full.set_index('id')
    full_list.append( full)
    test = pd.read_csv('{}/test.csv'.format(f))
    test['type'] = f.split('-')[0]
    test.set_index('id')
    test_list.append (test)
aggregated = pd.concat(agg_list)
if MERGE_OTHER:
    aggregated['label'] = aggregated.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)
    full['label'] = full.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)


print(aggregated.describe())
full = pd.concat(full_list)
test_questions = pd.concat(test_list)
aggregated = aggregated[aggregated['_unit_state'] == 'finalized']
aggregated.to_csv('train-data.csv')

def print_info(data):
    avg = pd.merge(data, full, suffixes=['_l', '_r'], on=['id'], how='left')[['id', 'text_readable_r', 'label:confidence', 'label_l', '_golden', 'label_r', '_trust','a','b']].sort_values(['label:confidence', 'label_l' ,'id'])
    renamed_avg = avg.rename(columns={'label_l' : 'assigned_by_crowdflower', 'label_r' : 'proposed_by_annotator'})
    return renamed_avg


           _unit_id  _trusted_judgments  label:confidence  label_gold
count  1.500000e+03         1500.000000       1500.000000         0.0
mean   1.559704e+09            3.084667          0.880948         NaN
std    1.642296e+07            0.278478          0.180800         NaN
min    1.543287e+09            3.000000          0.333300         NaN
25%    1.543287e+09            3.000000          0.682900         NaN
50%    1.559704e+09            3.000000          1.000000         NaN
75%    1.576122e+09            3.000000          1.000000         NaN
max    1.576122e+09            4.000000          1.000000         NaN


### Quality

In [4]:
conf = aggregated[['id','text_readable', 'label:confidence', 'label']]
display(conf[['label:confidence']].describe())
display(conf[['id', 'text_readable']].describe())

,label:confidence
count,1500.000000
mean,0.880948
std,0.180800
min,0.333300
25%,0.682900
50%,1.000000
75%,1.000000
max,1.000000


,id,text_readable
count,1500,1500
unique,1500,1500
top,A107951124,This first place finish earned 32 points for *...
freq,1,1


Sentences with a confidence <= 0.6. Either all three annotators gave a different answer or the annotators are not trustworthy enough.

In [5]:
worst_conf = conf[conf['label:confidence'] <= 0.6]
worst_conf.describe()

,label:confidence
count,66.000000
mean,0.391583
std,0.075834
min,0.333300
25%,0.340850
50%,0.353200
75%,0.384875
max,0.549900


Sentences with 0.60 > confidence < 0.75. Those sentences might introduce noise, but should be useable.

In [6]:
avg_conf = conf[(conf['label:confidence']< 0.75) & (conf['label:confidence']> 0.60)]
avg_conf.describe()

,label:confidence
count,410.000000
mean,0.670905
std,0.023905
min,0.608100
25%,0.656350
50%,0.668350
75%,0.682800
max,0.747600


Sentences with a confidence greater than 0.75

In [7]:
top_conf = conf[conf['label:confidence'] >= 0.80]
top_conf.describe()

,label:confidence
count,1009.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


All sentences which should contain enough information to train the system (Confidence >= 66%)

In [8]:
useable = conf[conf['label:confidence'] >= 0.67]
useable.describe()

,label:confidence
count,1213.000000
mean,0.948828
std,0.114426
min,0.670100
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [9]:
def class_summary(df):
    dc = defaultdict(list)
    diff = defaultdict(int)
    for index, row in df.iterrows():
        
        dc[row['id']].append(row['proposed_by_annotator'])
        dc[row['id']] = sorted(dc[row['id']])
        diff[row['id']] = len(set(dc[row['id']]))

    overview = pd.merge( pd.DataFrame(list(dc.items()), columns=['id', 'labels']) , pd.DataFrame(list(diff.items()), columns=['id', 'distinct']), on='id')
    certain = overview[overview['distinct'] > 1]
    return overview, certain
    #return 

class_summary(print_info(conf))[0].groupby('distinct').count()
len(class_summary(print_info(conf))[1])

320

### Hard Sentences

In [10]:
display(print_info(worst_conf.sample(20)).style.background_gradient(cmap=plt.cm.get_cmap('RdYlGn'),subset=['label:confidence']))
#merged = print_info(worst_conf)



merged.groupby(by=['text_readable_r']).proposed_by_annotator.describe()

,id,text_readable_r,label:confidence,assigned_by_crowdflower,_golden,proposed_by_annotator,_trust,a,b
0,S3005172629,"*virginia* Tech put together a solid drive, but §michigan$ forced a 37-yard field goal.",0.3364,WORSE,False,BETTER,0.9,virginia,michigan
1,S3005172629,"*virginia* Tech put together a solid drive, but §michigan$ forced a 37-yard field goal.",0.3364,WORSE,False,WORSE,0.9,virginia,michigan
2,S3005172629,"*virginia* Tech put together a solid drive, but §michigan$ forced a 37-yard field goal.",0.3364,WORSE,False,NONE,0.875,virginia,michigan
45,N2179423563,She hit a *golf* ball farther and a §tennis$ serve harder than most of them did.,0.3378,NONE,False,WORSE,0.8,golf,tennis
46,N2179423563,She hit a *golf* ball farther and a §tennis$ serve harder than most of them did.,0.3378,NONE,False,BETTER,0.8333,golf,tennis
47,N2179423563,She hit a *golf* ball farther and a §tennis$ serve harder than most of them did.,0.3378,NONE,False,NONE,0.8333,golf,tennis
6,G956609037,Holding a *pistol* steady is harder than a §rifle$.,0.3404,NONE,False,WORSE,0.8621,rifle,pistol
7,G956609037,Holding a *pistol* steady is harder than a §rifle$.,0.3404,NONE,False,BETTER,0.8333,rifle,pistol
8,G956609037,Holding a *pistol* steady is harder than a §rifle$.,0.3404,NONE,False,NONE,0.875,rifle,pistol
3,G1000253659,I agree with the above mentioned statement of getting a *wii* if you just want to have fun and look at mediocre graphics or better yet a §ps2$ which has tons of games with decent graphics and gameplay.,0.3443,WORSE,False,WORSE,0.8824,wii,ps2


NameError: name 'merged' is not defined

### Average Sentences

In [ ]:
print_info(avg_conf.sample(20)).style.background_gradient(cmap=plt.cm.get_cmap('RdYlGn'),subset=['label:confidence'])

### Easy Sentences

In [ ]:
print_info(top_conf.sample(20)).style.background_gradient(cmap=plt.cm.get_cmap('RdYlGn_r'),subset=['label:confidence'])

### All Sentences

In [ ]:
color_map = plt.cm.get_cmap('RdYlGn')
#plt.title('Confidence of Labeling')
plt.grid(axis='y', linestyle='dashed')
ticks = [0.3, 0.4, 0.5, 0.66, 0.7, 0.8, 0.9, 1]
plt.xticks(ticks)
n, bins, patches = plt.hist(
    aggregated['label:confidence'], bins=ticks, color='green')
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# scale values to interval [0,1]
col = bin_centers - min(bin_centers)
col /= max(col)

for c, p in zip(col, patches):
    plt.setp(p, 'facecolor', color_map(c))
plt.show()

In [ ]:
color_map = plt.cm.get_cmap('RdYlGn')
#plt.title('Confidence of Labeling')
plt.grid(axis='y', linestyle='dashed')
ticks = [0.0,  0.66, 1]
plt.xticks(ticks)
n, bins, patches = plt.hist(
    aggregated['label:confidence'], bins=ticks, color='green')
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# scale values to interval [0,1]
col = bin_centers - min(bin_centers)
col /= max(col)

for c, p in zip(col, patches):
    plt.setp(p, 'facecolor', color_map(c))
plt.show()

## Results

**Label Distribution**

In [ ]:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

def show_dist(labels):

    plt.figure(figsize=plt.figaspect(1))
    plt.pie(
        labels.value_counts().values,
        labels=labels.value_counts().keys().tolist(),
        radius=2,
        autopct=make_autopct(labels.value_counts().values))
    plt.show()


**All sentences**

In [ ]:
show_dist(aggregated['label'])

**Sentences with conf >= 66**

In [ ]:
show_dist(useable['label'])